In [1]:
import requests
from tqdm import tqdm
from os.path import join as oj
import tables, numpy
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi
from skimage import data
import pickle as pkl
from skimage.util import img_as_float
from sklearn import metrics
import h5py
from copy import deepcopy
from skimage.filters import gabor_kernel
import gabor_feats
from sklearn.linear_model import RidgeCV
import seaborn as sns
import numpy.linalg as npl
out_dir = '/scratch/users/vision/data/gallant/vim_2_crcns'
out_name = oj(out_dir, f'out_{dset}.h5')
from run import *

In [ ]:
brick = h5py.File(out_name, 'r')['data'][0]
grass = h5py.File(out_name, 'r')['data'][100]
gravel = h5py.File(out_name, 'r')['data'][300]
images = [brick, grass, gravel]


# Plot a selection of the filter bank kernels and their responses.
results = []
kernel_params = []
for theta in [0, 1]:
    theta = theta / 4. * np.pi
    for frequency in [0.1, 0.4]:
        kernel = gabor_kernel(frequency, theta=theta)
        params = 'theta=%d,\nfrequency=%.2f' % (theta * 180 / np.pi, frequency)
        kernel_params.append(params)
        results.append((kernel, [gabor_feats.calc_feats(img, kernel) for img in images])) # Save kernel and the power image for each image

fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(5, 6), dpi=200)
plt.gray()
axes[0][0].axis('off')


# Plot original images
for img, ax in zip(images, axes[0][1:]):
    ax.imshow(img)
    ax.axis('off')

# Plot Gabor kernel    
for label, (kernel, _), ax_row in zip(kernel_params, results, axes[1:]):
    ax = ax_row[0]
    ax.imshow(np.real(kernel))
    ax.set_ylabel(label, fontsize=7)
    ax.set_xticks([])
    ax.set_yticks([])

# Plot Gabor responses with the contrast normalized for each filter    
for label, (kernel, features), ax_row in zip(kernel_params, results, axes[1:]):
    ax = ax_row[2]
    ax.set_xticks([])
    ax.set_yticks([])
    vmin = np.min(features)
    vmax = np.max(features)
    for patch, ax in zip(features, ax_row[1:]):
        ax.imshow(patch, vmin=vmin, vmax=vmax)
        ax.axis('off')  

plt.tight_layout()
plt.show()